In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
#   Eventually this will be a demo notebook and go in the right place       #
#                                                                           #
#   For now it has scratch work for showing current branch capabilities     #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

In [1]:
from transformers import FastSpeech2ConformerConfig, FastSpeech2ConformerModel
import yaml
import argparse
from pathlib import Path

config_files = {'model_file': '/Users/connorhenderson/Desktop/fun/porting/from-pretrained/train.total_count.ave_5best.pth',
 'train_config': '/Users/connorhenderson/Desktop/fun/porting/from-pretrained/config.yaml'}

train_config_file = Path(config_files['train_config'])
with train_config_file.open("r", encoding="utf-8") as f:
    args = yaml.safe_load(f)
    args = argparse.Namespace(**args)
    

config = FastSpeech2ConformerConfig(**args.tts_conf['text2mel_params'])
model = FastSpeech2ConformerModel(config)

/Users/connorhenderson/Desktop/fun/huggingface/transformers/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/connorhenderson/Desktop/fun/huggingface/transformers/env/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
import torch

device = "cpu"

def load_model_weights(model):
    tts_task_state_dict = torch.load(config_files['model_file'], map_location=device)
    def filter_and_modify_keys(obj):
        new_obj = {}
        for key in obj:
            if "tts.generator.text2mel." in key:
                if "postnet" in key and("running" in key or "num_batches_tracked" in key):
                    continue
                if "encoder.embed.0.weight" in key:
                    new_obj["encoder.embed.weight"] = obj[key]
                    continue
                new_key = key.replace("tts.generator.text2mel.", "")
                new_obj[new_key] = obj[key]
        return new_obj
    text2mel_state_dict = filter_and_modify_keys(tts_task_state_dict)
    model.load_state_dict(text2mel_state_dict)

load_model_weights(model)
print(model)

FastSpeech2ConformerModel(
  (encoder): FastSpeech2ConformerEncoder(
    (embed): Embedding(78, 384, padding_idx=0)
    (pos_enc): RelPositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoders): ModuleList(
      (0): EncoderLayer(
        (self_attn): RelPositionMultiHeadedAttention(
          (linear_q): Linear(in_features=384, out_features=384, bias=True)
          (linear_k): Linear(in_features=384, out_features=384, bias=True)
          (linear_v): Linear(in_features=384, out_features=384, bias=True)
          (linear_out): Linear(in_features=384, out_features=384, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear_pos): Linear(in_features=384, out_features=384, bias=False)
        )
        (feed_forward): MultiLayeredConv1d(
          (w_1): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
          (w_2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
          (dropout): Dropout(p=0.2, inpla

In [5]:
import torch
from transformers import SpeechT5HifiGan

# Currently poor, but working input_ids -> saved wav file example

def write_example_to_file(model):
    # tokenized version of "test that this generates speech"
    batch = {'text': torch.tensor([ 4, 15,  6,  4,  9, 18,  4,  9, 12,  6, 40, 15,  3, 21, 47,  4,  6,  6,
        17, 27, 39])} 
    # with torch.no_grad():
    feat_gen = model.inference(**batch)
    
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
    output_dict = model.generate_speech(feat_gen=feat_gen, vocoder=vocoder)
    
    import soundfile
    file = './audios/test.wav'
    soundfile.write(file=file, data=output_dict["wav"].cpu().numpy(), samplerate=22050)

write_example_to_file(model)

end torch.Size([1, 21, 384])
end torch.Size([1, 180, 384])
